# chapter 2. Machine Learning Starting with Scikit-Learn

## Model Selection Module

### 학습 / 테스트 데이터 세트 분리 - train_test_split()

사이킷런의 **model_selection 모듈**은 학습 데이터와 테스트 데이터 세트를 분리하거나 교차 검증 분할 및 평가, 그리고 Estimator의 하이퍼 파라미터를 튜닝하기 위한 다양한 함수와 클래스를 제공한다. <br>

**학습 / 테스트 데이터 세트 분리 - train_test_split()** <br>

학습 데이터 세트로만 학습하고 예측하면, 이미 학습한 데이터 세트를 기반으로 예측하는 것이기 때문에 정확도가 100%이 나오는 상황이 발생한다. <br> 

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터 세트로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도 : ', accuracy_score(train_label, pred))

예측 정확도 :  1.0


사이킷런의 train_test_split()을 통해 **원본 데이터 세트에서 학습 및 테스트 데이터 세트를 쉽게 분리**할 수 있다. <br>

1. sklearn.model_selection 모듈에서 train_test_split()을 로드한다. <br>
2. train_test_split()은 첫번째 파라미터로 피처 데이터 세트, 두번째 파라미터로 레이블 데이터 세트를 입력받는다. <br>
3. 선택적으로 다음 파라미터를 입력받는다. 

|용어|설명|
|:------:|:---:|
|test_size|전체 데이터에서 테스트 데이터 세트 크기를 얼마로 샘플링 할 것인가를 결정한다. <br> 디폴트는 0.25, 즉 25%이다.|
|train_size|전체 데이터에서 학습용 데이터 세트 크기를 얼마로 샘플링 할 것인가를 결정한다. <br> test_size parameter를 통상적으로 사용하기 때문에 train_size는 잘 사용하지 않는다.|
|shuffle|데이터를 분리하기 전에 데이터를 미리 섞을지 결정한다. <br> 디폴트는 True이다. 데이터를 분산시켜서 좀 더 효율적인 학습 및 테스트 데이터 세트를 만드는데 사용한다.|
|random_state|random_state는 호출할 때마다 동일한 학습 / 테스트용 데이터 세트를 생성하기 위해 주어지는 난수 값이다. <br> train_test_split()은 호출 시 무작위로 데이터를 분리하므로 ramdon_state를 지정하지 않으면 수행할 때마다 다른 학습 / 테스트용 데이터를 생성한다.|

<br>

**train_test_split()의 반환 값은 튜플 형태**이다. <br>
순차적으로 학습용 데이터의 피처 데이터 세트, 테스트용 데이터의 피처 데이터 세트, 학습용 데이터의 레이블 데이터 세트, 테스트용 데이터의 레이블 데이터 세트가 반환된다. 

In [5]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

X_train, X_test, Y_train, Y_test = train_test_split(iris_data.data, iris_data.target, 
                                                   test_size = 0.3, random_state = 121)

In [6]:
dt_clf.fit(X_train, Y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도 : {0:.4f}'.format(accuracy_score(Y_test, pred)))

예측 정확도 : 0.9556


### 교차 검증

알고리즘을 학습시키는 학습 데이터와 이에 대한 예측 성능을 평가하기 위한 별도의 테스트용 데이터를 필요로 한다. <br>
이 방법은 **과적합(Overfitting)** 에 취약한 약점을 가진다. <br>
과적합은 **모델이 학습 데이터에만 과도하게 최적화**되어, **실제 예측**을 다른 데이터로 수행할 경우에는 **예측 성능이 과도하게 떨어지는 것**을 말한다. <br>
**테스트 데이터에만 최적의 성능을 발휘**할 수 있도록 편향되게 모델을 유도하는 경향인 것이다. <br>

교차 검증을 사용하게 되면 이 문제점을 개선할 수 있다. <br>
**교차 검증**은 **데이터 편중을 막기 위해서 별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행**하는 것이다. <br>
교차 검증을 사용하게 되면 모델 최적화를 더 쉽게 할 수 있다. <br>
대부분의 **ML 모델의 성능 평가는 교차 검증 기반으로 1차 평가를 한 뒤 최종적으로 테스트 데이터 세트에 적용해 평가하는 프로세스**이다. <br>
학습, 검증, 테스트 데이터 세트로 나눌 수 있다. 

<img src = 'image/Cross Validation.jpg' alt = 'Cross Validation' width='700' height='700'>

**K 폴드 교차 검증**

K 폴드 교차 검증은 가장 보편적으로 사용되는 교차 검증 기법이다. <br>
**K개의 데이터 폴드 세트를 만들어서 K번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행하는 방법**이다. 

<img src = 'image/KFold Cross Validation.jpg' alt = 'KFold Cross Validation' width='700' height='700'>

학습 데이터 세트와 검증 데이터 세트를 점진적으로 변경하면서 마지막 K번째까지 학습과 검증을 수행한다. <br>
**K개의 예측 평가를 구했으면 이를 평균내 K 폴드 평가 결과로 반영**한다. <br>
사이킷런에서는 K 폴드 교차 검증 프로세스를 구현하기 위해 **KFold**와 **StratifiedKFold** 클래스를 제공한다. 

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state = 156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits = 5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기 : ', features.shape[0])

붓꽃 데이터 세트 크기 :  150


KFold 객체는 split()을 호출하면 학습용 / 검증용 데이터로 분할할 수 있는 Index를 반환한다. 

In [10]:
n_iter = 0

# KFold 객체의 split()을 호출하면 폴드별 학습용, 검증용 테스트의 row Index를 array로 반환
for train_index, test_index in kfold.split(features):
    # kfold.split()으로 반환된 Index를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    Y_train, Y_test = label[train_index], label[test_index]
    # 학습 및 예측
    dt_clf.fit(X_train, Y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    # 반복 시마다 정확도 측정
    accuracy = np.round(accuracy_score(Y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 Index : {1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('평균 검증 정확도 : ', np.mean(cv_accuracy))

#1 교차 검증 정확도 : 1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#1 검증 세트 Index : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
#2 교차 검증 정확도 : 0.9667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#2 검증 세트 Index : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]
#3 교차 검증 정확도 : 0.8667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#3 검증 세트 Index : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]
#4 교차 검증 정확도 : 0.9333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#4 검증 세트 Index : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
#5 교차 검증 정확도 : 0.7333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#5 검증 세트 Index : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
평균 검증 정확도 :  0.9


**Stratified K 폴드** <br>

Stratified K 폴드는 **불균형한 분포도를 가진 레이블 데이터 집합을 위한 K 폴드 방식**이다. <br>
불균형한 분포도를 가진 레이블 데이터 집합은 **특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것**을 말한다. <br>
레이블 값으로 1이 특정 개별 반복별 학습 / 테스트 데이터 세트에는 상대적으로 많이 들어있고, 다른 반복 학습 / 테스트 데이터 세트에는 그렇지 못한 결과가 발생한다. <br>
우너본 데이터와 유사한 레이블 값의 분포를 학습 / 테스트 데이터 세트에도 유지하는 것이 매우 중요하다. <br>
**Stratified K 폴드는 원본 데이터의 레이블 분포를 먼저 고려한 뒤 이 분포와 동일하게 학습과 검증 데이터 세트를 분배**한다. 

In [11]:
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64